# Authors

- Ikram Kohil, 2019115
- Johnatan Gao, 2013298

# Taint Analysis
In this lab, we will be implementing taint analysis to ensure that all data coming from the outside (user inputs) do not reach critical parts of the system.

# 1. Implementation and tests

In this first part we will implement the teint analysis algorithm called: Possibly Teinted Definitions. We will then test the implementation on four examples in the folder **part_1**.

## 1.1 Possibly Tainted Definitions

According to class notes, there are a couple of variables/concepts that we need to define in order to implement the conceptual algorithm.

$$
in\_taintedDefs: V → \Rho(DEFS)
$$
$$
out\_taintedDefs: V → \Rho(DEFS)
$$

In this statement:

- V: This typically represents a set or type of variables within the context of a programming language or a system. Variables can be anything from integers, strings, objects, etc., depending on the language or system under consideration.
- P(DEFS): P represents the power set, which is the set of all subsets of a given set. DEFS is a set of definitions. So, P(DEFS) would be the power set of the set of definitions. In this context, it suggests that for each variable in V, there is a set of definitions (DEFS) associated with it, and in_taintedDefs maps each variable in V to a subset of its associated definitions.

$$
in\_taintedDefs(v) = \bigcup_{p \in preds(node)} out\_taintedDefs(p)
$$

This equation defines the behavior of the in_taintedDefs function. It states that the in_taintedDefs for a variable v is the union (⋃) of the out_taintedDefs for all predecessors (preds(node)) of a given node. In other words, to compute the in_taintedDefs for a variable at a particular node, you take the union of the out_taintedDefs for all nodes that flow into that node.

$$
tainted\_GEN[node] = 
\left\{
  \begin{array}{ll}
    \ {d_i \in defs[node]  | \exists(d_j, r_k) \in defRefChains | (r_k \in refs[node]), (d_j \in in\_tainted[node])} &\text{ if } (node \in EXPR)\ \\

    \ {d_i \in defs[node]} & \text{ if } (node \in SOURCES) \\

    \emptyset & \text{ if } (node \in FILTERS) \\
    
    \emptyset & \text{ if } (node \in SAFESET)
  \end{array}
\right.
$$

In this statement, we are trying to populate our GEN array. For each node in our CFG, we check for four conditions:
- If the node is part of the FILTERS set, then the tainted_gen for this node is empty
- If the node is part of the SAFESET set, then the tainted_gen for this node is empty as well
- If the node is part of the SOURCES set, then the tainted_gen for this node is the definition for this node
- If the node is part of the EXPR set (in other words, is an expression), then we will:
  - Take the defintion for this node, if it exists
  - Take the definition/reference pair for this node, if it exists
  - Create a pair of (reference, definition), if the reference is part of our reference set and definition is part of the in_tainted set for this node
  
In other words, 
1. Check if node is a definition: First, determine if the current node is a definition.

2. If the node is a definition and the right side is a source: If the node is indeed a definition, and its right side is a source, then mark the definition as tainted. This means that the definition corresponds to a value that is considered tainted or untrusted.

3. If the node is a definition and it corresponds to a filter or safe: If the node is a definition but corresponds to a filter or safe, then no action is taken. This implies that the definition is associated with a value that is considered safe or filtered, so it doesn't need further processing.

4. If the node is a definition and the right side is an expression: If the node is a definition and its right side is an expression, further investigation is needed.
Check if any references in the expression are tainted: Examine all references in the expression associated with the definition. If any of these references are tainted (i.e., marked as untrusted), then the definition is also considered tainted.

$$
tainted\_KILL[node] = {d_k | (var(d_k) = var(d_m)) ^ (d_m \in defs[node] )}
$$

In this statement, we are trying to populate the kill set. Essentially, everytime we detect a definition OR a redefinition, we must add it to our tainted_kill set for this particular node.

In [ ]:
import os
import json
from pathlib import Path
from code_analysis import CFGReader
from code_analysis import CFG

# Global variable - directory where cfg.json and .dot files generated by our code will be stored 
part1_output_directory = "output/part_1/"
part2_output_directory = "output/part_2/"

# Utility functions taken from TP1
def get_json_files(extension, directory):
   directory = Path(directory)
   return [str(file) for file in directory.rglob(extension)]

def create_output_file(filename, directory):
    # Check if output directory exists, if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Check if output file already exists, if so, delete and create new file
    file_path = os.path.join(directory, filename)
    if os.path.exists(file_path):
        os.remove(file_path)

    # Open in "append" mode to avoid overwriting the whole file after each modification
    return open(directory + filename, "a")

def get_filename_without_extension(file_path):
    file_basename = os.path.basename(file_path)
    filename_without_extension = file_basename.split('.')[0]
    return filename_without_extension

def close_output_file(file):
   file.close()

In [ ]:
from typing import Dict, List, Set


class TaintAnalysisAlgorithm:
    def __init__(self, filename):
        self.cfg = None
        self.filename = filename

        # Dictionnary containing all necessary parameters (safe, filter, etc)
        self.tainted_params: Dict[str, List[int]] = {} # Format: key = param_type(safe/filter/etc) value = node_ids arary

        # GEN and KILL dictionnaries
        ## Dictionnary containing all tainted_gen nodes for a specific node
        self.tainted_gen:  Dict[int, Set] = {} # Format (for all following dictionnaries): key = node_id, value = set of node_ids
        ## Dictionnary containing all tainted_kill nodes for a specific node
        self.tainted_kill: Dict[int, Set] = {}

        # IN and OUT dictionnaries
        ## Dictionnary containing all tainted_in nodes for a specific node
        self.tainted_in: Dict[int, Set] = {}
        ## Dictionnary containing all tainted_out nodes for a specific node
        self.tainted_out: Dict[int, Set] = {}

    def __init_tainted_params(self, taint_json_filename):
        # Read the file and initialize the appropriate parameters in a dictionnary
        taint_file = open(f'{taint_json_filename}.php.taint.json')
        params = json.load(taint_file)

        self.tainted_params = {
            'defs': params['defs'],
            'refs': params['refs'],
            'pairs': params['pairs'],
            'sinks': params['sinks'],
            'filters': params['filters'],
            'safes': params['safes'],
            'sources': params['sources']
        }

    def __get_expr_nodes(self, expr_node_id):
        # Simulate a do while in order to keep going down the cfg until all references/(nodes that arent operators) made in the expression are retrieved
        ref_nodes = []

        # do
        ref_node_id, expr_id = self.cfg.get_op_hands(expr_node_id)

        # Check if variable to not accidentally add constants like 1, 2, 3 etc.
        # if self.cfg.get_type(ref_node_id) == "Variable":
        #     ref_nodes.append(ref_node_id)

        while self.cfg.get_type(expr_id) == "BinOP":
            print("inside while")
            ref_node_id, expr_id = self.cfg.get_op_hands(expr_id)

            if self.cfg.get_type(ref_node_id) == "Variable":
                ref_nodes.append(ref_node_id)

        # Since we're out of the binop while, then the last right side node is part of the expression
        ref_nodes.append(expr_id)

        return ref_nodes    

    def __is_binOp_equal(self, node_id: int):
        return self.cfg.get_type(node_id) == "BinOP" and self.cfg.get_image(node_id) == "="

    def get_nodes_tainted_gen(self, node_id):
        gens_value = self.tainted_gen.get(node_id)
        if gens_value is None:
            self.tainted_gen[node_id] = set()

        ## To determine if the definition is tainted, we need to check the right side of the definition, and we need to check EACH node involved (each reference)
        ## Ex: For definition x = y + z + w +1, we need to check y, z and w. If AT LEAST one of them is tainted, then the definition is tainted
        ## To do so, we need to check for BinOP nodes
        if self.__is_binOp_equal(node_id):
            var_node_id, expr_node_id = self.cfg.get_op_hands(node_id)

             # EXPR part of the algo
            if self.cfg.get_type(expr_node_id) == "BinOP":
                # First condition to check is if the defined variable is part of the tainted definitions
                if var_node_id in self.tainted_params['defs']:
                    self.tainted_gen[node_id].add(var_node_id)
            
                # Second condition to check is separated in two parts, and we need all the references made in the expression
                ref_nodes = []
                ref_nodes = self.__get_expr_nodes(expr_node_id)
                for ref_id in ref_nodes:
                    for (definition, reference) in self.tainted_params['pairs']:
                        # First we need to see if the reference is part of the given tainted def/ref pairs
                        # Second we need to see of the associated reference is part of the tainted definitions, either by checking the given tainted pairs,
                        ## or by checking if it has been tainted during the code execution (hence why we check the tainted_in)
                        if ref_id == reference and (definition in self.tainted_params['defs'] or definition in self.tainted_in[expr_node_id]):
                            self.tainted_gen[node_id].add(var_node_id)

        elif node_id in self.tainted_params['sources']:
            self.tainted_gen[node_id].add(node_id)
            
        ## If part of filter or safe, then not tainted (in which case, skip)
        elif node_id in self.tainted_params['filters']:
            self.tainted_gen[node_id] = set()
        elif node_id in self.tainted_params['safes']:
            self.tainted_gen[node_id] = set()
    
    def get_nodes_tainted_kill(self, node_id):
        if self.__is_binOp_equal(node_id):
            var_node_id = self.cfg.get_op_hands(node_id)[0]

            # According to the algorithm, if the defined variable is tainted, then we kill it
            if var_node_id in self.tainted_params['defs']:
                kills_value = self.tainted_kill.get(node_id)
                if kills_value is None:
                    self.tainted_kill[node_id] = {var_node_id}
                else:
                    kills_value.add(var_node_id)

    def get_taint_analysis(self, taint_json_filename, cfg: CFG):
        self.cfg = cfg

        # Start by initializing the relevant parameters for the analysis in order to populate the gen and kill dictionnaries for each node
        self.__init_tainted_params(taint_json_filename)

        # Retrieve the nodeSet. The algorithm we have to implement cannot be done recursively like we usually do 
        ## At least we found it simpler to do in an iterative manner, so as to follow the given algorithm as closely as possible
        ## So the nodeSet here is the list of all nodes in the cfg
        node_set = self.cfg.get_node_ids()

        for node_id in node_set:
            # Initialize the gen and kill dictionnaries for the current node
            self.get_nodes_tainted_gen(node_id)
            self.get_nodes_tainted_kill(node_id)

            # Initialize the in and out dictionnaries for the current node
            ins_value = self.tainted_in.get(node_id)
            if ins_value is None:
                self.tainted_in[node_id] = set()

            outs_value = self.tainted_out.get(node_id)
            if outs_value is None:
                self.tainted_out[node_id] = set()

        changes = True
        old_out: Dict[int, Set] = {}

        while changes:
            changes = False

            for node_id in node_set:
                # First, we get the node's predecessor
                ## We noticed that in the case of CallEnd nodes, the get_parents doesnt return anything, but we found the fucntion get_call_begin
                ## which gives us the corresponding CallBegin node, which is the parent
                predecessors = [self.cfg.get_call_begin(node_id)] if self.cfg.get_type(node_id) == 'CallEnd' else self.cfg.get_parents(node_id)

                # Then, then in of the current node is composed of the out of each of those predecessors
                for pred in predecessors:
                    # Union allows us to add all the content of the out set at once without iterating over it
                    self.tainted_in[node_id] = self.tainted_in[node_id].union(self.tainted_out[pred])
                
                old_out[node_id] = self.tainted_out[node_id]

                # IN - KILL (default value to empty set in case node not found in kill, for error handling)
                in_kill_difference = self.tainted_in.get(node_id).difference(self.tainted_kill.get(node_id, set()))
                # GEN union (IN - KILL)
                self.tainted_out[node_id] = self.tainted_gen[node_id].union(in_kill_difference)

                if self.tainted_out[node_id] != old_out[node_id]:
                    changes = True

In [ ]:
# Prepare output file and reader
cfg_reader = CFGReader()

def get_refs_defs_pairs(directory):
    part1_output_directory = "output/part_1/"
    part_1_output_file = create_output_file("part_1_output_file.txt", part1_output_directory)

    # Retrieve filenames of all cfg in the specified directory
    cfgFilenames = get_json_files('*.cfg.json', directory)

    # Iterate over the filenames array once to visit all cfgs
    for filename in cfgFilenames:
        taint_filename =  get_filename_without_extension(filename)
        print(filename)

        # Load cfg in memory
        cfg = cfg_reader.read_cfg(filename)

        analyser = TaintAnalysisAlgorithm(filename)
        analyser.get_taint_analysis(directory_to_analyze + taint_filename, cfg)

        # Print output
        part_1_output_file.write(f"------------------------ File: {filename} ------------------------\n")
        for node in cfg.get_node_ids():
            print(f"{node}: , IN: {analyser.tainted_in[node]}, OUT: {analyser.tainted_out[node]}")

    
    close_output_file(part_1_output_file)

directory_to_analyze = "../part_1/"
get_refs_defs_pairs(directory_to_analyze)